In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel, AutoConfig
import tensorflow as tf
import torch
import tensorflow as tf
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
import itertools as it

/home/mats/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-03 12:21:06.438618: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-03 12:21:06.538648: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-03 12:21:06.684541: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-03 12:21:06.685060: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operati

In [2]:
config = AutoConfig.from_pretrained("model/")
model = AutoModel.from_pretrained("model/", config=config, from_tf=True)
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", return_tensors="tf")


All TF 2.0 model weights were used when initializing RobertaModel.

All the weights of RobertaModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaModel for predictions without further training.
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
model.eval()

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(130, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

In [4]:
names = pd.read_csv('data/names.csv').iloc[:5,:]
tweets = pd.read_csv('data/toxicity_en.csv')

In [5]:
# Twitter removes mentions in their preprocessing. However, for constructing the examples, I want to remove the mentions as well.
mention_re = re.compile(r"(\B@\S+)", re.IGNORECASE)
tweets['clean_text'] = [mention_re.sub("MENTION", tweet) for tweet in tweets['text']]

In [6]:
check_names = names['Republicans'].to_list()
check_names.extend(names['Democrats'].to_list())
check_names_split = [x.split(" ")[-1] for x in check_names]
check_names_split.append('Democrat')
check_names_split.append('Republican')
check_names_lower = [x.lower() for x in check_names_split]

In [7]:
names_check = []
for tweet in tweets['text']:
  if any(name in tweet.lower() for name in check_names_lower):
    names_check.append(False)
  else:
    names_check.append(True)

In [8]:
tweets

,text,is_toxic,clean_text
0,"Elon Musk is a piece of shit, greedy capitalis...",Toxic,"Elon Musk is a piece of shit, greedy capitalis..."
1,The senile credit card shrill from Delaware ne...,Toxic,The senile credit card shrill from Delaware ne...
2,He does that a lot -- makes everyone look good...,Toxic,He does that a lot -- makes everyone look good...
3,F*ck Lizzo,Toxic,F*ck Lizzo
4,Epstein and trump were best buds!!! Pedophiles...,Toxic,Epstein and trump were best buds!!! Pedophiles...
...,...,...,...
995,My maternal abuelita taught me how to make pla...,Not Toxic,My maternal abuelita taught me how to make pla...
996,Funnily enough I was looking online last week ...,Not Toxic,Funnily enough I was looking online last week ...
997,I can't bear how nice this is.\n \n I guess it...,Not Toxic,I can't bear how nice this is.\n \n I guess it...
998,Going to buy a share of Tesla just to ensure i...,Not Toxic,Going to buy a share of Tesla just to ensure i...


In [9]:
tweets['name_flag'] = names_check
tweets = tweets[tweets['name_flag'] == True]

In [10]:
tweets.shape

(871, 4)

In [11]:
target = 'I am NAME'

In [12]:
democrat = [target.replace('NAME', name) for name in names['Democrats']]
republican = [target.replace('NAME', name) for name in names['Republicans']]
neutral = [target.replace('NAME', name) for name in names['Neutral']]

In [13]:
toxic_tweets = tweets['clean_text'][tweets['is_toxic']=='Toxic'].sample(n=100, random_state=2).tolist()
non_toxic_tweets = tweets['clean_text'][tweets['is_toxic']=='Not Toxic'].sample(n=100, random_state=2).tolist()

In [14]:
def get_pooled_output(example: str):
  with torch.no_grad():
    array = model(**tokenizer(example, return_tensors='pt',padding=True, truncation=True), output_hidden_states=True).pooler_output
    return array.cpu().detach().numpy()

In [15]:
democrat_vec = [get_pooled_output(x) for x in democrat]
republican_vec = [get_pooled_output(x) for x in republican]
neutral_vec = [get_pooled_output(x) for x in neutral]

In [16]:
toxic_vec = [get_pooled_output(x) for x in toxic_tweets]
non_toxic_vec = [get_pooled_output(x) for x in non_toxic_tweets]

In [17]:
# Compute s(w,A,B)
def mean_cos(target, attribute_a, attribute_b):
  mean_sim_a = np.mean([cosine_similarity(target.reshape(1, -1), a) for a in attribute_a])
  mean_sim_b = np.mean([cosine_similarity(target.reshape(1, -1), b) for b in attribute_b])
  return mean_sim_a - mean_sim_b

In [18]:
def compute_test_stat(target_set_x, target_set_y, attribute_a, attribute_b):
  test_stat_x = np.sum([mean_cos(t, attribute_a, attribute_b) for t in target_set_x])
  test_stat_y = np.sum([mean_cos(t, attribute_a, attribute_b) for t in target_set_y])
  return test_stat_x - test_stat_y

In [19]:
def effect_size(target_set_x, target_set_y, attribute_a, attribute_b):
  effect_size_x = np.mean([mean_cos(t, attribute_a, attribute_b) for t in target_set_x])
  effect_size_y = np.mean([mean_cos(t, attribute_a, attribute_b) for t in target_set_y])

  target_set_union = np.concatenate((np.array(target_set_x), np.array(target_set_y)))
  std_union = np.std([mean_cos(t, attribute_a, attribute_b) for t in target_set_union])

  return (effect_size_x - effect_size_y) / std_union



In [20]:
def compute_p_val(X, Y, A, B):
    total_true, total, convergence = 0, 0, [] 
    n_samples = 10000
    size = len(X)
    observed_stat = compute_test_stat(X, Y, A, B)    
    XY = np.concatenate((X,Y))
    for _ in range(n_samples):
        np.random.shuffle(XY)
        Xi = XY[:size]
        permuted_stat = compute_test_stat(Xi, Y, A, B)
        if permuted_stat > observed_stat:
            total_true +=1
        elif permuted_stat == observed_stat:
            total_true +=1
        total +=1
        convergence.append(total_true/total)

    return total_true/total, convergence

In [22]:
effect_size_dem_rep = effect_size(democrat_vec, republican_vec, toxic_vec, non_toxic_vec)
effect_size_dem_neu = effect_size(democrat_vec, neutral_vec, toxic_vec, non_toxic_vec)
effect_size_rep_neu = effect_size(republican_vec, neutral_vec, toxic_vec, non_toxic_vec)

p_val_dem_rep, convergence_dem_rep = compute_p_val(democrat_vec, republican_vec, toxic_vec, non_toxic_vec)
p_val_dem_neu, convergence_dem_neu = compute_p_val(democrat_vec, neutral_vec, toxic_vec, non_toxic_vec)
p_val_rep_neu, convergence_rep_neu = compute_p_val(republican_vec, neutral_vec, toxic_vec, non_toxic_vec)


test_stat_dem_rep = compute_test_stat(democrat_vec, republican_vec, toxic_vec, non_toxic_vec)
test_stat_dem_neu= compute_test_stat(democrat_vec, neutral_vec, toxic_vec, non_toxic_vec)
test_stat_rep_neu = compute_test_stat(republican_vec, neutral_vec, toxic_vec, non_toxic_vec)

effect_sizes = [effect_size_dem_rep, effect_size_dem_neu, effect_size_rep_neu]
p_values = [p_val_dem_rep, p_val_dem_neu, p_val_rep_neu]
test_stats = [test_stat_dem_rep, test_stat_dem_neu, test_stat_rep_neu]

In [ ]:
results = pd.DataFrame({'effect_size': effect_sizes, 'p_value': p_values, 'test_stat': test_stats}, index = ['Democrat Republican', 'Democrat Neutral', 'Republican Neutral'])
results

In [ ]:
results.round(4).to_csv("data/seat_results.csv")